### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable
import io
import time

### Set Global Date Variable
Allows specification of which source CSV to pull and helps to ensure unique playlists are created

In [2]:
pullDate = "20210227"

### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [3]:
TOKEN = "BQBqLV6bFQ_tMpjYU4Ng1BGBnxmvU-bF6xBnHWdgh0frXQcdD-d1HCSf3hvXJQTI1h_8reg4CvojQ0mx2_AiOXoRRgIqKQs39yw0yU0KtDsqF8_pywksBieevYxzGXr7PgRK9NkXQNdiOPGQCnTdXNq63wiVbHkOsJ8mNTYt8uPV-NbUn85ilMBLTumkLg4bO2sZ9gIBUo7EtB02Kws2qf_647g1JG7ZOZ8"

### Header Construction

In [4]:
header = {"Authorization": "Bearer {}".format(TOKEN)}

### Request Construction
> Remember that Spotify will return a max of 20 objects by default, so artists with lots of albums might not truly be returning the most recent

In [5]:
artist = "bastille"
artist = artist.replace(" ", "%20")
artist = "artist:%22" + artist + "%22"
# album = "Bad Blood"
# album = album.replace(" ", "%20")
# album = "album:%22" + album + "%22"
# queryParam = album + artist
target = "https://api.spotify.com/v1/search?q={}&type=album".format(artist)
request = requests.get(target, headers = header).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=artist%3A%22bastille%22&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https://i.scdn.co/image/a

### Function Accepts CSV as Input and Pulls New Albums

Testing out artist searches that resulted in errors

In [11]:
newArtist = "L'FREAQ"
newArtist = newArtist.replace(" ", "%20")
newArtist = newArtist.replace("&", "%26")
newArtist = newArtist.replace("'", "%27")
newArtist = newArtist.replace(".", "%2E")
newArtist = "artist:%22" + newArtist + "%22"
target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
print(target)
request = requests.get(target, headers = header).json()
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
print(len(df.index))
print(df)
# paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
# converter = jsontable.converter()
# converter.set_paths(paths)
# resultTable = converter.convert_json(request)
# df = pd.DataFrame(data=resultTable)
# df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
# df = df.drop(df.index[0])
# df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
# newest = pd.DataFrame(data=df.iloc[[0]])

https://api.spotify.com/v1/search?q=artist:%22L%27FREAQ%22&type=album
1
    0           1       2             3
0  ID  Album Name  Tracks  Release Date


In [6]:
url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/New%20Music%20Source%20Files/iTunes%20Alternative%20{}.csv".format(pullDate)
# url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/sampleArtists.csv"
fetch = requests.get(url).content
artists = pd.read_csv (io.StringIO(fetch.decode('utf-8')))
artists.style

newAlbums = pd.DataFrame()
errors = []

def pullAlbums(artistDF):
    
    global newAlbums
    global errors
    
    for index, row in artistDF.iterrows():
        # Wait five seconds to avoid 20 calls/minute limit on iTunes API
        # time.sleep(5)
        newArtist = row["Artist"]
        newArtist = newArtist.replace(" ", "%20")
        newArtist = newArtist.replace("&", "%26")
        newArtist = newArtist.replace("'", "%27")
        newArtist = newArtist.replace(".", "%2E")
        newArtist = "artist:%22" + newArtist + "%22"
        target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
        request = requests.get(target, headers = header).json()
        paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        resultTable = converter.convert_json(request)
        df = pd.DataFrame(data=resultTable)
        df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
        if len(df.index) > 1:
            df = df.drop(df.index[0])
            df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
            newest = pd.DataFrame(data=df.iloc[[0]])
            newAlbums = newAlbums.append(newest)
        else:
            errors.append(row["Artist"])
            continue

    print(newAlbums)
    # newAlbums.style
    # Would prefer to have the output styled in this manner but am getting "style is not supported for non-unique indices"
    print(errors)
    
pullAlbums(artists)

                  Album ID                                  Album Name Tracks  \
5   7aRJMS2Jj9O21X09ES1nus                            1 Last Cigarette      2   
3   0ES5xVtEDOETW7dzhG270N            Abstinence Blues Mixhell Remixes      4   
2   78AMuxyO4J6ZRvmHQYLtSX                  All The Colors Of The Moon      1   
5   1bqMcZybfmAbCK8Dzdxjhp                   All These Perfect Crosses     20   
3   3uXf4k1CQWimgDDRZWBNZ9                                      Anthem      1   
..                     ...                                         ...    ...   
5   474p3hlvsesqSXUuCI7rYI                                  Free Trial     19   
6   2XB3HRzRuNWvBeSNTg53JP                       The Shadow I Remember     11   
5   6EhVPKV9UrF9kQ268zB3G6                                       Debra      1   
4   0X9oLn2APMvGOIsogM1TSp  The Exciting Sounds of Menahan Street Band     14   
7   4gkQEP3ETiZQgRoXYUdrhI                                         YES      1   

   Release Date  
5    2021

In [10]:
display(newAlbums)

,Album ID,Album Name,Tracks,Release Date
5,7aRJMS2Jj9O21X09ES1nus,1 Last Cigarette,2,2021-02-26
3,0ES5xVtEDOETW7dzhG270N,Abstinence Blues Mixhell Remixes,4,2021-02-26
2,78AMuxyO4J6ZRvmHQYLtSX,All The Colors Of The Moon,1,2021-02-26
5,1bqMcZybfmAbCK8Dzdxjhp,All These Perfect Crosses,20,2021-02-26
3,3uXf4k1CQWimgDDRZWBNZ9,Anthem,1,2021-02-26
2,0VaHzMHMKNxdtiPCnVt3dK,Apartment 24,2,2021-02-26
18,3cnXQQ2fghgnvdJeNKGIVw,Auf zur Illumination!,9,2021-02-26
7,7ivMLt1kj3to6Olb8jLtom,Banane Bleue,10,2021-02-26
4,5xD1JEquZrAlkpUp1l8SGD,Bassically (Acoustic),1,2021-02-26
4,5MaMJwKXyfLEv0f0mjyxlZ,Brighter Lighting,10,2021-02-26


### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [5]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style

,Album ID,Album Name,Tracks,Release Date
2,5G6oMu9zNW2acdV0lqzI3L,All This Bad Blood,25,2013-01-01


### Create Spotify Playlist
> Be careful here because Spotify will let you create playlists with the same name and description, assigning it a new playlist ID

In [7]:
userID = "t1e947vjw65dme71utp690f50"
createPlaylistTarget = "https://api.spotify.com/v1/users/{}/playlists".format(userID)
payload = {"name": pullDate, "description": "Created from Jupyter notebook", "public": False}
# payload = {"name": "Test Playlist", "description": "Created from Jupyter notebook", "public": False}
payload = json.dumps(payload, indent=2)
createPlaylistPOST = requests.post(createPlaylistTarget, headers = header, data = payload).json()
print(json.dumps(createPlaylistPOST, indent=2))

{
  "collaborative": false,
  "description": "Created from Jupyter notebook",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/5ktEPZyPAUpX1TizpbbvnH"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/5ktEPZyPAUpX1TizpbbvnH",
  "id": "5ktEPZyPAUpX1TizpbbvnH",
  "images": [],
  "name": "20210227",
  "owner": {
    "display_name": "Patrick",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
    },
    "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
    "id": "t1e947vjw65dme71utp690f50",
    "type": "user",
    "uri": "spotify:user:t1e947vjw65dme71utp690f50"
  },
  "primary_color": null,
  "public": false,
  "snapshot_id": "MSxiNDY0MDA1Yzk3MmUzNWE2MTk1MTkxYTEyMjViYzg0Y2U2NjFmODVl",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/5ktEPZyPAUpX1TizpbbvnH/tracks",
    "items": [],
    "limit": 100,
    "next": null,
    "offset":

Retrieve User Information

In [19]:
getUserTarget = "https://api.spotify.com/v1/users/{}".format(userID)
getUser = requests.get(getUserTarget, headers = header).json()
print(json.dumps(getUser, indent=2))

{
  "display_name": "Patrick",
  "external_urls": {
    "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
  },
  "followers": {
    "href": null,
    "total": 1
  },
  "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
  "id": "t1e947vjw65dme71utp690f50",
  "images": [
    {
      "height": null,
      "url": "https://i.scdn.co/image/ab6775700000ee85804e93a20b51ae91923b1a51",
      "width": null
    }
  ],
  "type": "user",
  "uri": "spotify:user:t1e947vjw65dme71utp690f50"
}


### Fetch Track IDs for Album
Store tracks in a dataframe and potentially sort on popularity (not seeing clear metadata to support this) 
Probably makes sense to add to the playlist in this loop so that it's adding chunks of tracks per album and any kind of error handling can be associated with the album.

In [8]:
playlistID = "5ktEPZyPAUpX1TizpbbvnH"
newTracks = pd.DataFrame()
popVals = []

def assembleTracks(albumDF, playlist):
    
    global newTracks
    global popVals
    
    # Assemble request to add the album's tracks to the target playlist which is passed as function parameter
    addTracksTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist)
    
    # For each album object, fetch its tracks and add their IDs and names to global dataframe
    for index, row in albumDF.iterrows():
        # Wait five seconds to avoid rate limit
        # time.sleep(5)
        newAlbumID = row["Album ID"]
        fetchTrackTarget = "https://api.spotify.com/v1/albums/{}/tracks".format(newAlbumID)
        fetchTrack = requests.get(fetchTrackTarget, headers = header).json()
        # parse JSON for appropriate track metadata to store
        paths = [{"$.items.artists.name":"Track ID"}, {"$.items.id":"Track ID"}, {"$.items.name":"Track Name"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        trackTable = converter.convert_json(fetchTrack)
        df = pd.DataFrame(data=trackTable)
        df.columns = ["Artist", "Track ID", "Track Name"]
        df = df.drop(df.index[0])
        newTracks = newTracks.append(df)

        # Assemble track URIs to add to payload for request to add tracks to playlist
        trackURIs = []
        
        # For each track in the album add URI to list, then add set of tracks to Spotify playlist
        for index, row in df.iterrows():
            selectedTrack = row["Track ID"]
            selectedTrackURI = "spotify:track:{}".format(selectedTrack)
            trackURIs.append(selectedTrackURI)
            print(trackURIs)
            
            # Retrieve popularity for each track
            
            getPopularityTarget = "https://api.spotify.com/v1/tracks/{}".format(selectedTrack)
            trackDetails = requests.get(getPopularityTarget, headers = header).json()
            
            # Parse popularity values and add to global list that will be appended to newTracks dataframe
            
            paths = [{"$.popularity":"Popularity"}]
            converter = jsontable.converter()
            converter.set_paths(paths)
            popularityColumn = converter.convert_json(trackDetails)
            tempPop = pd.DataFrame(data=popularityColumn)
            tempPop = tempPop.drop(tempPop.index[0])
            newVal = tempPop.iloc[0][0]
            popVals.append(newVal)
        
        # Assemble payload to add album's worth of tracks to Spotify playlist
        payload = {"uris": trackURIs, "position":"null"}
        payload = json.dumps(payload, indent=2)
        
        # Execute request to update playlist
        addTracks = requests.post(addTracksTarget, headers = header, data = payload).json()
    
    # Full logic has run, adding accumulated popularity values as a column in newTracks dataframe
    newTracks["Popularity"] = popVals    
    print(newTracks)
        
assembleTracks(newAlbums, playlistID)

['spotify:track:07cZs4Bn5aMHsQMFZ7MOui']
['spotify:track:07cZs4Bn5aMHsQMFZ7MOui', 'spotify:track:0SdV67igfBWmSLnzU2jLV5']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:6H7fCmmDG2mjcVAGTYkqwC']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch', 'spotify:track:4DmfmpiHbd9nP5RYddtAGg']
['spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:6H7fCmmDG2mjcVAGTYkqwC', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch', 'spotify:track:4tEsIgbAcG4rSKq8CBZ0Ch', 'spotify:track:4DmfmpiHbd9nP5RYddtAGg', 'spotify:track:4DmfmpiHbd9nP5RYddtAGg']
['spotify:track:6H7fCmmDG2mjcVAG

['spotify:track:7bHof8BRJCA4pq11dLUMVy']
['spotify:track:7hMfB8zmqDlO6qvJQPoGIO']
['spotify:track:7hMfB8zmqDlO6qvJQPoGIO', 'spotify:track:1I6ddtoHieOs1HDDCpysiM']
['spotify:track:18Kr8xsfirW52Vh14oiqby']
['spotify:track:18Kr8xsfirW52Vh14oiqby', 'spotify:track:05GUGU56OmAdjCeCjVlXI4']
['spotify:track:18Kr8xsfirW52Vh14oiqby', 'spotify:track:05GUGU56OmAdjCeCjVlXI4', 'spotify:track:25HJvlexhneh0nKd8a3RKx']
['spotify:track:18Kr8xsfirW52Vh14oiqby', 'spotify:track:05GUGU56OmAdjCeCjVlXI4', 'spotify:track:25HJvlexhneh0nKd8a3RKx', 'spotify:track:3Eu5OoLf6WgfwLbkORw2Ry']
['spotify:track:18Kr8xsfirW52Vh14oiqby', 'spotify:track:05GUGU56OmAdjCeCjVlXI4', 'spotify:track:25HJvlexhneh0nKd8a3RKx', 'spotify:track:3Eu5OoLf6WgfwLbkORw2Ry', 'spotify:track:4YgoOvNHCmYCa5vV4NY7cZ']
['spotify:track:18Kr8xsfirW52Vh14oiqby', 'spotify:track:05GUGU56OmAdjCeCjVlXI4', 'spotify:track:25HJvlexhneh0nKd8a3RKx', 'spotify:track:3Eu5OoLf6WgfwLbkORw2Ry', 'spotify:track:4YgoOvNHCmYCa5vV4NY7cZ', 'spotify:track:5b24Hc14pQEFJJly

['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:3BKES4esGk2NbxbKwA4T54']
['spotify:track:3BKES4esGk2NbxbKwA4T54', 'spotify:track:3xCyrh9Ok77WrvF9Q0LcX2']
['spotify:track:3BKES4esGk2NbxbKwA4T54', 'spotify:track:3xCyrh9Ok77WrvF9Q0LcX2', 'spotify:track:3xCyrh9Ok77WrvF9Q0LcX2']
['spotify:track:2MjzsLWwEBqnJAdXtDlaz1']
['spotify:track:1VywDfHbI99GrKzsG1aa3a']
['spotify:track:1VywDfHbI99GrKzsG1aa3a', 'spotify:track:7FOT5Wc7GX3KSSFq422c10']
['spotify:track:1VywDfHbI99GrKzsG1aa3a', 'spotify:track:7FOT5Wc7GX3KSSFq422c10', 'spotify:track:6FELEsEwduJaChoZtDp2EM']
['spotify:track:1VywDfHbI99GrKzsG1aa3a', 'spotify:track:7FOT5Wc7GX3KSSFq422c10', 'spotify:track:6FELEsEwd

['spotify:track:2UjOpW79eok1epLPkZ0dtJ']
['spotify:track:5eDvWpMdQHtN8NXwojaAH1']
['spotify:track:5eDvWpMdQHtN8NXwojaAH1', 'spotify:track:2jtP62bQLvNaouZ1afbt6P']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt', 'spotify:track:5WHjHldMRq7kQJIVKcrz4h']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt', 'spotify:track:5WHjHldMRq7kQJIVKcrz4h', 'spotify:track:18fddUe7cbQrFzDT3Byugb']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt', 'spotify:track:5WHjHldMRq7kQJIVKcrz4h', 'spotify:track:18fddUe7cbQrFzDT3Byugb', 'spotify:track:18fddUe7cbQrFzDT3Byugb']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt', 'spotify:track:5WHjHldMRq7kQJIVKcrz4h', 'spotify:track:18fddUe7cbQrFzDT3Byugb', 'spotify:track:18fddUe7cbQrFzDT3Byugb', 'spotify:track:7CBtao1Qxa2xZ4BTHZm1YV']
['spotify:track:4mDYBNL8d5LudfT2JI3aJt', 'spotify:track:5WHjHldMRq7kQJIVKcrz4h', 'spotify:track:18fddUe7cbQrFzDT3Byugb', 'spotify:track:18fddUe7cbQrFzDT3Byugb', 'spotify:track:7CBtao1Qxa2xZ4BTHZm1YV', 'spotify:track:5bFMkWOiyxaXMV1k

['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:None']
['spotify:track:1vj41KsWucXdGfbUyDxH98']
['spotify:track:1dthu6dMsSfRe2k1nBTL36']
['spotify:track:2mkqPfdKt1Z0sifkN4QI3E']
['spotify:track:2mkqPfdKt1Z0sifkN4QI3E', 'spotify:track:0NCAzFp2rhSd0tAiYYPy2l']
['spotify:track:2mkqPfdKt1Z0sifkN4QI3E', 'spotify:track:0NCAzFp2rhSd0tAiYYPy2l', 'spotify:track:1cj1u18eCjlHYimlSVPVl7']
['spotify:track:2mkqPfdKt1Z0sifkN4QI3E', 'spotify:track:0NCAzFp2rhSd0tAiYYPy2l', 'spotify:track:1cj1u18eCjlHYimlSVPVl7', 's

['spotify:track:0CyDC1ZuvWIa1aHMOoToF4']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ', 'spotify:track:5rT9tDSw6XXcObyuqM0Wkm']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ', 'spotify:track:5rT9tDSw6XXcObyuqM0Wkm', 'spotify:track:1cq2SfJrVYyJH7jF0oP957']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ', 'spotify:track:5rT9tDSw6XXcObyuqM0Wkm', 'spotify:track:1cq2SfJrVYyJH7jF0oP957', 'spotify:track:1K2Cr1NzGl6QwZj1mMZJyO']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ', 'spotify:track:5rT9tDSw6XXcObyuqM0Wkm', 'spotify:track:1cq2SfJrVYyJH7jF0oP957', 'spotify:track:1K2Cr1NzGl6QwZj1mMZJyO', 'spotify:track:28r3HKNPxgwoJeNQ9PyckI']
['spotify:track:0CyDC1ZuvWIa1aHMOoToF4', 'spotify:track:4vLsZ18xKvBE5cgQh4zDRZ', 'spotify:track:5rT9tDSw6XXcObyuqM0Wkm', 'spotify:track:1cq2SfJrVYyJH7jF0o

In [9]:
pd.set_option("display.max_rows", None)
display(newTracks)

,Artist,Track ID,Track Name,Popularity
1,The Band CAMINO,07cZs4Bn5aMHsQMFZ7MOui,1 Last Cigarette,21.0
2,The Band CAMINO,0SdV67igfBWmSLnzU2jLV5,Roses,7.0
1,Meatraffle,6H7fCmmDG2mjcVAGTYkqwC,Abstinence Blues - Mixhell Extended Remix,0.0
2,Mixhell,6H7fCmmDG2mjcVAGTYkqwC,Abstinence Blues - Mixhell Extended Remix,0.0
3,Meatraffle,4tEsIgbAcG4rSKq8CBZ0Ch,Abstinence Blues - Mixhell Edit,0.0
4,Mixhell,4tEsIgbAcG4rSKq8CBZ0Ch,Abstinence Blues - Mixhell Edit,0.0
5,Meatraffle,4DmfmpiHbd9nP5RYddtAGg,Abstinence Blues - Mixhell Remix Instrumental,0.0
6,Mixhell,4DmfmpiHbd9nP5RYddtAGg,Abstinence Blues - Mixhell Remix Instrumental,0.0
7,Meatraffle,3Dor8lZGuKy6QQHPo2JBQz,Abstinence Blues,0.0
1,Pale Puma,1Poq0EDAHoqFt5APN7IrZe,All The Colors Of The Moon,0.0


### Add Tracks to a Playlist
POST https://api.spotify.com/v1/playlists/{playlist_id}/tracks  
Need the playlist ID and a list of track URIs  

> Folded this into the assembleTracks function, so the logic below is now just a reference for making a single call

In [9]:
playlistID = "25NbFuQM1kDleGz1bjGJ2x"
addTrackTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlistID)
payload = {"uris": ["spotify:track:57As4lhkpp6tB422daefKK","spotify:track:3ePeWP6f8ccQtF7wayoNrD"], "position":"null"}
payload = json.dumps(payload, indent=2)
addTrack = requests.post(addTrackTarget, headers = header, data = payload).json()
print(json.dumps(addTrack, indent=2))

{
  "snapshot_id": "MTIsMzBmODE0NzUwOGVlNGJmZWZmZWYyMjExOGI2NzYyNjQzNzY5ZDExNA=="
}
